# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
% matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation
from madminer.plotting import plot_fisher_information_contours_2d

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_2/'

In [13]:
sample_dir = base_dir + 'data/samples/wgamma/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma/'
result_dir = base_dir + 'data/results/wgamma/'

## FisherInformation instance

In [5]:
fisher = FisherInformation(sample_dir + 'samples.h5', debug=True)

10:48  
10:48  ------------------------------------------------------------
10:48  |                                                          |
10:48  |  MadMiner v2018.10.22                                    |
10:48  |                                                          |
10:48  |           Johann Brehmer, Kyle Cranmer, and Felix Kling  |
10:48  |                                                          |
10:48  ------------------------------------------------------------
10:48  
10:48  Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma/samples.h5
10:48  Found 2 parameters:
10:48     cWWW (LHA: dim6 1, maximal power in squared ME: (2,), range: (-0.1, 0.1))
10:48     cWWWtilde (LHA: dim6 2, maximal power in squared ME: (2,), range: (-0.1, 0.1))
10:48  Found 6 benchmarks:
10:48     sm: cWWW = 0.00e+00, cWWWtilde = 0.00e+00
10:48     morphing_basis_vector_1: cWWW = -9.39e-02, cWWWtilde = 0.07
10:48     morphing_basis_vector_2: cWWW = 0.

## Truth-level info

In [16]:
fisher_info_parton = fisher.calculate_fisher_information_full_truth(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_parton.npy', fisher_info_parton)


## Rate-only info

In [15]:
fisher_info_xsec = fisher.calculate_fisher_information_rate(
    theta=[0.,0.],
    luminosity = 300000.
)

np.save(result_dir + 'information_xsec.npy', fisher_info_xsec)


## Full Fisher info (ML)

In [6]:
fisher_info_ensemble_mean, fisher_info_ensemble_cov = fisher.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_all',
    unweighted_x_sample_file = sample_dir + 'test/x_test.npy'
)

np.save(result_dir + 'information_mean_full.npy', fisher_info_ensemble_mean)
np.save(result_dir + 'information_covariance_full.npy', fisher_info_ensemble_cov)

10:49  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/ensemble.json
10:49    Found ensemble with 10 estimators and expectations [[ 0.00634092  0.06508537]
 [ 0.02007575  0.02556378]
 [ 0.03975902 -0.01588391]
 [-0.02189546  0.02988753]
 [-0.01648937 -0.00555408]
 [-0.00935855  0.00249242]
 [ 0.04448106 -0.01657623]
 [ 0.0275156  -0.01794838]
 [-0.05467386  0.020074  ]
 [-0.00354157 -0.00803116]]
10:49  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_0_settings.json
10:49    Found method sally, 27 observables, 2 parameters, (50,) hidden layers, tanh activation function, features None
10:49  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_all/estimator_0_state_dict.pt
10:49  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/dat

## Info in observables (ML)

In [20]:
fisher_info_ensemble_mean, fisher_info_ensemble_cov = fisher.calculate_fisher_information_full_detector(
    theta=[0.,0.],
    luminosity=300000.,
    model_file=model_dir + 'sally_ensemble_deltaphi',
    unweighted_x_sample_file = sample_dir + 'test/x_test.npy'
)

np.save(result_dir + 'information_mean_deltaphi.npy', fisher_info_ensemble_mean)
np.save(result_dir + 'information_covariance_deltaphi.npy', fisher_info_ensemble_cov)

11:54  Loading ensemble setup from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/ensemble.json
11:54    Found ensemble with 10 estimators and expectations [[ 0.01108136  0.01226927]
 [-0.02025858 -0.03946016]
 [-0.00724483 -0.00818603]
 [-0.00711934  0.01605911]
 [-0.00019295  0.00029173]
 [ 0.0075682   0.01714741]
 [ 0.00396435 -0.00194454]
 [ 0.00240622 -0.00631471]
 [ 0.00656226  0.00237199]
 [-0.00282423 -0.00644198]]
11:54  Loading settings from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/estimator_0_settings.json
11:54    Found method sally, 1 observables, 2 parameters, (50,) hidden layers, tanh activation function, features [25]
11:54  Loading state dictionary from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/models/wgamma/sally_ensemble_deltaphi/estimator_0_state_dict.pt
11:54  Loading settings from /Users/johannbrehmer/work/projects/madminer/dibo

## Histogram info

In [17]:
filenames = ['met', 'ptl', 'pta', 'deltaphi_la']
observables = ['et_miss', 'pt_l1', 'pt_a1', 'deltaphi_l1_a1']
obs_mins = [0., 0., 0., 0.]
obs_maxs = [500., 500., 500., 6.]
bins = 25

In [18]:
for filename, obs, obs_min, obs_max in zip(filenames, observables, obs_mins, obs_maxs):
    info = fisher.calculate_fisher_information_hist1d(
                theta=[0.,0.],
                luminosity=300000.,
                observable=obs,
                nbins=bins,
                histrange=(obs_min,obs_max)
           )
    
    np.save(result_dir + 'information_histo_{}.npy'.format(filename), info)
    

/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:780: RuntimeWarning: divide by zero encountered in double_scalars
  fisher_info.append(luminosity / sigma[i_event] * np.tensordot(dsigma.T[i_event], dsigma.T[i_event], axes=0))
/Users/johannbrehmer/work/projects/madminer/madminer/madminer/fisherinformation.py:780: RuntimeWarning: invalid value encountered in multiply
  fisher_info.append(luminosity / sigma[i_event] * np.tensordot(dsigma.T[i_event], dsigma.T[i_event], axes=0))
